In [1]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token:  ········


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-25 22:03:30.577558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 22:03:30.597648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 22:03:30.603805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register facto

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

/var/tmp/pbs.38363.sophia-pbs-01.lab.alcf.anl.gov/ipykernel_3404111/454465620.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  falcon = HuggingFaceHub(


In [5]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

## Zero-shot prompting

In [6]:
zero_shot_prompt = "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1."
kwargs = {"max_length": 1000}
response_zero_shot = get_completion_falcon(f"The odd numbers in this group add up to an even number: {zero_shot_prompt}", **kwargs)
print(response_zero_shot)


invoking llm at Huggingface Hub
#### User: 
The odd numbers in this group add up to an even number: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Response from falcon-7b-instruct: 
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Output: 
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Output: 
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Output: 
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Output: 
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Output: 
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

#### Output: 
The odd numbers in this group add up t

## Few-shot prompting

In [7]:
few_shot_prompt = """Examples:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: True

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""
response_few_shot = get_completion_falcon(few_shot_prompt, **kwargs)
print(response_few_shot)


invoking llm at Huggingface Hub
#### User: 
Examples:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: True

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:

#### Response from falcon-7b-instruct:

The odd numbers in the given group do not add up to an even number.

The statement is true.


## Chain-of-thought Prompting

In [8]:
cot_prompt = """Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A: First, select the odd numbers: (15, 5, 13, 7, 1). Next, add the numbers (15 + 5 + 13 + 7 + 1 = 41). 41 is an odd number, so the answer is False."""
response_cot = get_completion_falcon(cot_prompt, **kwargs)
print(response_cot)


invoking llm at Huggingface Hub
#### User: 
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A: First, select the odd numbers: (15, 5, 13, 7, 1). Next, add the numbers (15 + 5 + 13 + 7 + 1 = 41). 41 is an odd number, so the answer is False.

#### Response from falcon-7b-instruct:

Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A: First, select the odd numbers: (15, 5, 13, 7, 1). Next, add the numbers (15 + 5 + 13 + 7 + 1 = 41). 41 is an odd number, so the answer is False.


## Few-shot Chain-of-thought Prompting

In [9]:
few_shot_COT_prompt = """
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""

response_few_shot_cot = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(response_few_shot_cot)

invoking llm at Huggingface Hub
#### User: 

You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from falcon-7b-instruct:

The answer is False.

The template of the quest

## Play with temperature parameter

In [10]:
response_high_temp = get_completion_falcon(few_shot_COT_prompt, host="remote", temperature=1.0, **kwargs)
response_low_temp = get_completion_falcon(few_shot_COT_prompt, host="remote", temperature=0.2, **kwargs)

print("High Temperature Response:\n", response_high_temp)
print("\nLow Temperature Response:\n", response_low_temp)


invoking llm at Huggingface Hub
invoking llm at Huggingface Hub
High Temperature Response:
 #### User: 

You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from falcon-7

- Zero-shot is quick but less reliable.
- Few-shot improves response accuracy by example-based learning.
- Chain-of-thought enhances reasoning but increases verbosity.
- Few-shot chain-of-thought is the most comprehensive and accurate but computationally demanding


Low temperatures render outputs that are predictable and repetitive. Conversely, high temperatures encourage LLMs to produce more random, creative responses.